# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
#dropping null values
spaceship.dropna(inplace=True)
#taking only first character of values in column cabin
spaceship['Cabin'] = spaceship['Cabin'].str[0]
#dropping passenger ID & name
spaceship.drop(['PassengerId', 'Name'], axis=1, inplace=True)
spaceship.head(30)
#casting cryosleep to boolean to avoid doing drop dummies on this
spaceship['CryoSleep'] = spaceship['CryoSleep'].astype(bool)
#same approach for VIP
spaceship['VIP'] = spaceship['VIP'].astype(bool)
#applying the drop dummies method on 3 columns identified above
spaceship_dd = pd.get_dummies(spaceship, columns=['HomePlanet', 'Cabin', 'Destination'], drop_first=True)
spaceship_dd.head(30)
#Displaying final df
spaceship_dd.head(30)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Europa,HomePlanet_Mars,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False,True,False,True,False,False,False,False,False,False,False,True
1,False,24.0,False,109.0,9.0,25.0,549.0,44.0,True,False,False,False,False,False,False,True,False,False,False,True
2,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,True,False,False,False,False,False,False,False,False,False,True
3,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,True,False,False,False,False,False,False,False,False,False,True
4,False,16.0,False,303.0,70.0,151.0,565.0,2.0,True,False,False,False,False,False,False,True,False,False,False,True
5,False,44.0,False,0.0,483.0,0.0,291.0,0.0,True,False,False,False,False,False,False,True,False,False,True,False
6,False,26.0,False,42.0,1539.0,3.0,0.0,0.0,True,False,False,False,False,False,False,True,False,False,False,True
8,False,35.0,False,0.0,785.0,17.0,216.0,0.0,True,False,False,False,False,False,False,True,False,False,False,True
9,True,14.0,False,0.0,0.0,0.0,0.0,0.0,True,True,False,True,False,False,False,False,False,False,False,False
11,False,45.0,False,39.0,7295.0,589.0,110.0,124.0,True,True,False,True,False,False,False,False,False,False,False,False


In [4]:
#creating the features & target dfs
X = spaceship_dd.drop('Transported', axis=1)
y = spaceship_dd['Transported']
#perform train / test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [5]:
#feature scaling
normalizer = MinMaxScaler()

normalizer.fit(X_train)

MinMaxScaler()

In [6]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [7]:
#bagging model is most accurate
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bagging_class = BaggingClassifier(DecisionTreeClassifier(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [8]:
bagging_class.fit(X_train_norm, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=20),
                  max_samples=1000, n_estimators=100)

- Evaluate your model

In [9]:
from sklearn.metrics import accuracy_score

pred = bagging_class.predict(X_test_norm)

accuracy = accuracy_score(y_test, pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

correct_predictions = (y_test == pred).sum()
print(f"Correct Predictions: {correct_predictions} out of {len(y_test)}")

Accuracy: 78.74%
Correct Predictions: 1041 out of 1322


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [11]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Define the model with `estimator` instead of `base_estimator`
bagging_class = BaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42)

# Define the parameter grid
param_grid = {
    'estimator__max_depth': [10, 20, 30],      # max depth of Decision Tree
    'n_estimators': [50, 100, 150],            # number of trees in Bagging
    'max_samples': [500, 1000, 1500]           # number of samples per tree
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=bagging_class, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit GridSearchCV to find the best parameters
grid_search.fit(X_train_norm, y_train)

# Retrieve the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Parameters found by GridSearchCV:", best_params)
print("Best Model:", best_model)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best Parameters found by GridSearchCV: {'estimator__max_depth': 10, 'max_samples': 500, 'n_estimators': 150}
Best Model: BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=10),
                  max_samples=500, n_estimators=150, random_state=42)


- Run Grid Search

In [15]:
#re run the model after hyperparamter tuning : Best Model: BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=10),max_samples=500, n_estimators=150, random_state=42)
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bagging_class = BaggingClassifier(
    estimator=DecisionTreeClassifier(max_depth=10, random_state=42),  # random_state for base estimator
    n_estimators=150,
    max_samples=500,
    random_state=42)

In [16]:
bagging_class.fit(X_train_norm, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=10,
                                                   random_state=42),
                  max_samples=500, n_estimators=150, random_state=42)

- Evaluate your model

In [17]:
#previous results w/o hyper parameters tuning
#Accuracy: 78.74%
#Correct Predictions: 1041 out of 1322


from sklearn.metrics import accuracy_score

pred = bagging_class.predict(X_test_norm)

accuracy = accuracy_score(y_test, pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

correct_predictions = (y_test == pred).sum()
print(f"Correct Predictions: {correct_predictions} out of {len(y_test)}")

Accuracy: 79.12%
Correct Predictions: 1046 out of 1322


In [ ]:
#improved results